In [ ]:
from datetime import datetime, timezone
import ape
import pickle
from ape import chain, networks, Contract
import pandas as pd

In [ ]:
def flatten(logs):
    flat = []
    for log in logs:
        t = log.event_arguments
        t["block_number"] = log.block_number
        t["transaction_hash"] = log.transaction_hash.hex()
        flat.append(t)
    return pd.DataFrame(flat)

networks.parse_network_choice("ethereum:mainnet:geth").__enter__()

In [ ]:
aztec_connect = Contract("rollup.aztec.eth")
connect_deployed = 14923081

if False:
    ac_deposits = flatten(aztec_connect.Deposit.range(
        start_or_stop=connect_deployed, 
        stop = chain.blocks.height, 
        block_page_size = 10000
    ))
    pickle.dump(ac_deposits, open("ac_deposits.p", "wb"))
ac_deposits = pickle.load(open("ac_deposits.p", "rb"))

In [ ]:
# Take only stuff that has a blocknumber > x
cutoff_block_start = 0
cutoff_block_end = 15147113

cutoff_eth = 1e18
cutoff_dai = 2000e18

in_time = ac_deposits[
    (ac_deposits["block_number"] > cutoff_block_start) & 
    (ac_deposits["block_number"] < cutoff_block_end)
]
eligible = in_time.drop(["block_number", "transaction_hash"], axis=1)
eligible = eligible.groupby(["assetId", "depositorAddress"]).sum()
eligible = eligible.reset_index(level=[0, 1])
eligible = eligible[
    ((eligible["assetId"] == 0) & (eligible["depositValue"] >= cutoff_eth)) |
    ((eligible["assetId"] == 1) & (eligible["depositValue"] >= cutoff_dai))
]

winner = eligible.sample().values[0]

In [ ]:
print("For the timeframe {0} (block {1}) to {2} (block {3}) we have:".format(
    datetime.fromtimestamp(networks.active_provider.get_block(cutoff_block_start).timestamp, timezone.utc), 
    cutoff_block_start,
    datetime.fromtimestamp(networks.active_provider.get_block(cutoff_block_end).timestamp, timezone.utc),
    cutoff_block_end
))
print("There is a total of {0} users with deposits (eth or dai)".format(len(in_time)))
print("There is a total of {0} users with deposits (>{1} dai)".format(
    len(eligible[eligible["assetId"] == 1]),
    cutoff_dai / 1e18
))
print("There is a total of {0} users with deposits (>{1} eth)".format(
    len(eligible[eligible["assetId"] == 0]),
    cutoff_eth / 1e18
))
print('The winner is {0}, and have deposited {1} {2}'.format(
    winner[1], 
    winner[2] / 1e18, 
    "dai" if winner[0] == 1 else "eth"
))